In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# read in data
traffic_data = pd.read_csv("data/master.csv")
overflow_rates = pd.read_csv("data/overflow_rates.csv")

# Autonomous Vehicle Lanes

Next we will measure the impact of setting out specific lanes for autonomous traffic. With 100% automation in these lanes, we can use the maximum flow increase ratio for these autonomous lanes, and the regular free flow for the other lanes.

In [5]:
av_ratio = overflow_rates.loc[10, 'flow_increase_ratio']

# First try to find 1 lane data for highway 5 in decreasing direction
av_lanes = [0, 1, 2, 3, 4]
# find flow increases for 1 lane

def find_flow_for_lanes(traffic_data, av_lane_count):
    colname1 = f'flow_decr_lane_{av_lane_count}'
    colname2 = f'flow_incr_lane_{av_lane_count}'
    for i in range(len(traffic_data)):
        total_lanes_decr = traffic_data.loc[i, 'Number of Lanes DECR MP direction']
        total_lanes_incr = traffic_data.loc[i, 'Number of Lanes INCR MP direction']
        
        if av_lane_count <= total_lanes_decr: 
            av_lanes_decr = av_ratio * (traffic_data.loc[i, 'free_flow_decr'] * (av_lane_count / total_lanes_decr))
            hum_lanes_decr = traffic_data.loc[i, 'free_flow_decr'] * ((total_lanes_decr - av_lane_count) / total_lanes_decr)
        else:
            av_lanes_decr = traffic_data.loc[i, 'free_flow_decr'] * av_ratio
            hum_lanes_decr = 0
        
        
        if av_lane_count <= total_lanes_incr: 
            av_lanes_incr = av_ratio * (traffic_data.loc[i, 'free_flow_incr'] * (av_lane_count / total_lanes_incr))
            hum_lanes_incr = traffic_data.loc[i, 'free_flow_incr'] * ((total_lanes_incr - av_lane_count) / total_lanes_incr)
        else:
            av_lanes_incr = traffic_data.loc[i, 'free_flow_incr'] * av_ratio
            hum_lanes_incr = 0
        
        traffic_data.loc[i, colname1] = av_lanes_decr + hum_lanes_decr
        traffic_data.loc[i, colname2] = av_lanes_incr + hum_lanes_incr
    
for lanes in av_lanes:
    find_flow_for_lanes(traffic_data, lanes)

traffic_data



        
# find overflow percentage for 1 lane



# find overflow sum for for 1 lane 

,Unnamed: 0,Unnamed: 0.1,Route_ID,startMilepost,endMilepost,Average daily traffic counts Year_2015,"RteType (IS= Interstate, SR= State Route)",Number of Lanes DECR MP direction,Number of Lanes INCR MP direction,Unnamed: 7,...,flow_decr_lane_0,flow_incr_lane_0,flow_decr_lane_1,flow_incr_lane_1,flow_decr_lane_2,flow_incr_lane_2,flow_decr_lane_3,flow_incr_lane_3,flow_decr_lane_4,flow_incr_lane_4
0,0,0,5,100.93,101.87,65000,IS,3,3,NaN,...,4800.0,4800.0,5180.884641,5180.884641,5561.769282,5561.769282,5942.653923,5942.653923,5942.653923,5942.653923
1,1,1,5,101.87,103.17,85000,IS,3,3,NaN,...,4800.0,4800.0,5180.884641,5180.884641,5561.769282,5561.769282,5942.653923,5942.653923,5942.653923,5942.653923
2,2,2,5,103.17,103.42,108000,IS,3,3,NaN,...,4800.0,4800.0,5180.884641,5180.884641,5561.769282,5561.769282,5942.653923,5942.653923,5942.653923,5942.653923
3,3,3,5,103.42,104.81,101000,IS,3,3,NaN,...,4800.0,4800.0,5180.884641,5180.884641,5561.769282,5561.769282,5942.653923,5942.653923,5942.653923,5942.653923
4,4,4,5,104.81,105.63,144000,IS,3,4,NaN,...,4800.0,6400.0,5180.884641,6780.884641,5561.769282,7161.769282,5942.653923,7542.653923,5942.653923,7923.538564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,219,520,9.60,9.72,95000,SR,2,2,NaN,...,3200.0,3200.0,3580.884641,3580.884641,3961.769282,3961.769282,3961.769282,3961.769282,3961.769282,3961.769282
220,220,220,520,9.72,11.35,68000,SR,2,2,NaN,...,3200.0,3200.0,3580.884641,3580.884641,3961.769282,3961.769282,3961.769282,3961.769282,3961.769282,3961.769282
221,221,221,520,11.35,12.05,91000,SR,2,2,NaN,...,3200.0,3200.0,3580.884641,3580.884641,3961.769282,3961.769282,3961.769282,3961.769282,3961.769282,3961.769282
222,222,222,520,12.05,12.38,78000,SR,2,2,NaN,...,3200.0,3200.0,3580.884641,3580.884641,3961.769282,3961.769282,3961.769282,3961.769282,3961.769282,3961.769282


## 

In [7]:
lane_data = pd.DataFrame()
lane_data['lanes'] = av_lanes

def compute_lane_percentage(traffic_data, lanes):
    lane_traffic_decr, lane_traffic_incr = 0, 0
    total_traffic = traffic_data['rush_hour_decr_mp'].sum()
    for i in range(len(traffic_data)):
        total_lanes_decr = traffic_data.loc[i, 'Number of Lanes DECR MP direction']
        total_lanes_incr = traffic_data.loc[i, 'Number of Lanes INCR MP direction']
        
        if lanes <= total_lanes_decr: 
            lane_traffic_decr = lane_traffic_decr + (traffic_data.loc[i, 'rush_hour_decr_mp'] * lanes/total_lanes_decr)
        else:
            lane_traffic_decr = lane_traffic_decr + traffic_data.loc[i, 'rush_hour_decr_mp']
        
        if lanes <= total_lanes_incr: 
            lane_traffic_incr = lane_traffic_incr + (traffic_data.loc[i, 'rush_hour_incr_mp'] * lanes/total_lanes_incr)
        else:
            lane_traffic_incr = lane_traffic_incr + traffic_data.loc[i, 'rush_hour_incr_mp']
    
    return (lane_traffic_decr/total_traffic),(lane_traffic_incr/total_traffic) 

lane_perc_incr = []
lane_perc_decr = []
for lanes in av_lanes:
    per_decr, per_incr = compute_lane_percentage(traffic_data, lanes)
    lane_perc_decr.append(per_decr)
    lane_perc_incr.append(per_incr)
    

lane_data['lane_perc_incr'], lane_data['lane_perc_decr'] = lane_perc_incr, lane_perc_decr
 
lane_data

,lanes,lane_perc_incr,lane_perc_decr
0,0,0.000000,0.000000
1,1,0.317557,0.317557
2,2,0.635114,0.635115
3,3,0.900916,0.892740
4,4,0.983987,0.986690


In [9]:
def compute_overflow_percentage(t_data, flow_rate_col, free_flow_col):
    count = 0
    for i in range(len(t_data)):
        if t_data.loc[i, flow_rate_col] > t_data.loc[i, free_flow_col]:
                count = count + 1

    return count/len(t_data)  

def compute_sum_overflow(t_data, flow_rate_col, free_flow_col):
    overflow_sum = 0
    for i in range(len(t_data)):
        if t_data.loc[i, flow_rate_col] > t_data.loc[i, free_flow_col]:
                overflow_sum = overflow_sum + (t_data.loc[i, flow_rate_col] - t_data.loc[i, free_flow_col])

    return overflow_sum  

def overflow_sum_percentage(t_data, flow_rate_col, free_flow_col):
    return compute_sum_overflow(t_data, flow_rate_col, free_flow_col) / t_data[flow_rate_col].sum()
    

compute_overflow_percentage(traffic_data, 'rush_hour_decr_mp', 'flow_decr_lane_1')

0.5267857142857143

In [10]:
overflow_perc_decr = []
overflow_perc_incr = []
overflow_sum_decr = []
overflow_sum_incr = []

for lanes in av_lanes:
    col1, col2 = f'flow_decr_lane_{lanes}', f'flow_incr_lane_{lanes}'
    
    perc_decr = compute_overflow_percentage(traffic_data, 'rush_hour_decr_mp', col1)
    perc_incr = compute_overflow_percentage(traffic_data, 'rush_hour_incr_mp', col2)
    overflow_perc_decr.append(perc_decr)
    overflow_perc_incr.append(perc_incr)
    
    sum_decr = overflow_sum_percentage(traffic_data, 'rush_hour_decr_mp', col1)
    sum_incr = overflow_sum_percentage(traffic_data, 'rush_hour_incr_mp', col2)
    overflow_sum_decr.append(sum_decr)
    overflow_sum_incr.append(sum_incr)

lane_data['overflow_perc_decr'] = overflow_perc_decr
lane_data['overflow_perc_incr'] = overflow_perc_incr
lane_data['overflow_sum_decr'] = overflow_sum_decr
lane_data['overflow_sum_incr'] = overflow_sum_incr

lane_data  

,lanes,lane_perc_incr,lane_perc_decr,overflow_perc_decr,overflow_perc_incr,overflow_sum_decr,overflow_sum_incr
0,0,0.000000,0.000000,0.602679,0.602679,0.171353,0.172864
1,1,0.317557,0.317557,0.526786,0.526786,0.132713,0.134008
2,2,0.635114,0.635115,0.441964,0.437500,0.100530,0.101705
3,3,0.900916,0.892740,0.401786,0.397321,0.077264,0.077939
4,4,0.983987,0.986690,0.339286,0.339286,0.070667,0.072717


In [11]:
routes = [5, 405, 90, 520]

for route in routes:
    overflow_perc_decr = []
    overflow_perc_incr = []
    overflow_sum_decr = []
    overflow_sum_incr = []
    lane_perc_decr = []
    lane_perc_incr = []
    
    route_data = traffic_data[traffic_data.Route_ID == route]
    route_data = route_data.reset_index()
    
    for lanes in av_lanes:
        col1, col2 = f'flow_decr_lane_{lanes}', f'flow_incr_lane_{lanes}'

        perc_decr = compute_overflow_percentage(route_data, 'rush_hour_decr_mp', col1)
        perc_incr = compute_overflow_percentage(route_data, 'rush_hour_incr_mp', col2)
        overflow_perc_decr.append(perc_decr)
        overflow_perc_incr.append(perc_incr)

        sum_decr = overflow_sum_percentage(route_data, 'rush_hour_decr_mp', col1)
        sum_incr = overflow_sum_percentage(route_data, 'rush_hour_incr_mp', col2)
        overflow_sum_decr.append(sum_decr)
        overflow_sum_incr.append(sum_incr)
        
        per_decr, per_incr = compute_lane_percentage(traffic_data, lanes)
        lane_perc_decr.append(per_decr)
        lane_perc_incr.append(per_incr)

    lane_data[f'overflow_perc_decr_{route}'] = overflow_perc_decr
    lane_data[f'overflow_perc_incr_{route}'] = overflow_perc_incr
    lane_data[f'overflow_sum_decr_{route}'] = overflow_sum_decr
    lane_data[f'overflow_sum_incr_{route}'] = overflow_sum_incr
    lane_data[f'lane_perc_decr_{route}'] = lane_perc_decr
    lane_data[f'lane_perc_incr_{route}'] = lane_perc_incr


lane_data

,lanes,lane_perc_incr,lane_perc_decr,overflow_perc_decr,overflow_perc_incr,overflow_sum_decr,overflow_sum_incr,overflow_perc_decr_5,overflow_perc_incr_5,overflow_sum_decr_5,...,overflow_sum_decr_90,overflow_sum_incr_90,lane_perc_decr_90,lane_perc_incr_90,overflow_perc_decr_520,overflow_perc_incr_520,overflow_sum_decr_520,overflow_sum_incr_520,lane_perc_decr_520,lane_perc_incr_520
0,0,0.000000,0.000000,0.602679,0.602679,0.171353,0.172864,0.644444,0.644444,0.157043,...,0.062490,0.062360,0.000000,0.000000,0.200000,0.200000,0.052083,0.052083,0.000000,0.000000
1,1,0.317557,0.317557,0.526786,0.526786,0.132713,0.134008,0.548148,0.548148,0.120537,...,0.030983,0.030918,0.317557,0.317557,0.200000,0.200000,0.025032,0.025032,0.317557,0.317557
2,2,0.635114,0.635115,0.441964,0.437500,0.100530,0.101705,0.466667,0.459259,0.089690,...,0.016771,0.016736,0.635115,0.635114,0.066667,0.066667,0.009428,0.009428,0.635115,0.635114
3,3,0.900916,0.892740,0.401786,0.397321,0.077264,0.077939,0.414815,0.407407,0.064066,...,0.006356,0.006343,0.892740,0.900916,0.066667,0.066667,0.009428,0.009428,0.892740,0.900916
4,4,0.983987,0.986690,0.339286,0.339286,0.070667,0.072717,0.318519,0.318519,0.055458,...,0.006356,0.006343,0.986690,0.983987,0.066667,0.066667,0.009428,0.009428,0.986690,0.983987


In [12]:
lane_data.to_csv("data/lane_data.csv")
traffic_data.to_csv("data/master.csv")
overflow_rates.to_csv("data/overflow_rates.csv")